<a href="https://colab.research.google.com/github/skyqi/LangChain/blob/main/langchain_zhipui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2024-6-12

## 在LangChain内使用Google AI聊天模型
https://python.langchain.com/v0.2/docs/integrations/chat/google_generative_ai/#gemini-prompting-faqs

## 学习了使用BaseChatMessageHistory它通常用于记录用户和聊天系统之间的交互历史，以便系统能够根据上下文提供更准确的响应。

BaseChatMessageHistory 类的实现可能会因不同的对话系统和库而有所不同。在某些系统中，它可能只是一个简单的消息列表，而在其他系统中，它可能包含更复杂的结构和功能，比如支持对话状态管理、用户档案信息集成等。

## 还可以问AI：
我想知道如何将`BaseChatMessageHistory`用于用户档案管理。

## 从基类如 BaseChatMessageHistory 扩展的类可能会包括以下几种：


1，SimpleChatMessageHistory：一个简单的实现，可能会将聊天消息历史存储为一个列表。


2，PersistentChatMessageHistory：一个实现，它不仅存储聊天消息历史，还提供了将其保存到持久存储（如数据库）的功能。

3，FilteredChatMessageHistory：一个实现，它可以过滤聊天消息历史，例如只保留最近的消息或特定类型的信息。

4，ThreadedChatMessageHistory：一个实现，它支持多线程或异步对话，并能够正确管理不同线程或会话的历史。

5，UserProfileChatMessageHistory：一个实现，它结合了用户档案管理和聊天消息历史，可以提供个性化的聊天体验。


In [1]:
!pip install --upgrade --quiet  langchain-google-genai pillow
!pip install langchain langchain_community
!pip install --upgrade httpx httpx-sse PyJWT

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00


In [3]:
from langchain_community.chat_models import ChatZhipuAI
import os
import getpass
os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()

model = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)

··········


# 创建多人对话的BaseChatMessageHistory

In [ ]:

from langchain.agents import initialize_agent, Tool

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage



store = {}
# get_session_history 一直在保存。缺点容易引起输入token过大
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)
config = {"configurable": {"session_id": "abc2"}}
response = with_message_history.invoke(
    [HumanMessage(content="我的名字叫黑金")],
    config=config,
)

response = with_message_history.invoke(
    [HumanMessage(content="我早上吃的是油条")],
    config=config,
)

response = with_message_history.invoke(
    [HumanMessage(content="我中午吃的是米饭")],
    config=config,
)


response

··········


AIMessage(content='米饭是中国饮食中非常常见的主食，提供能量，对身体有益。搭配一些蔬菜和肉类，可以组成一顿营养均衡的午餐。希望你的午餐美味又满足！如果你想要分享你的食谱或者有其他问题，随时欢迎交流。', response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 82, 'total_tokens': 133}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-5cd99980-b56b-4af8-a536-18b1400406ce-0')

RunnableWithMessageHistory，可用于任何的chain中添加对话历史，将以下之一作为输入

（1）一个BaseMessage序列

（2）一个dict，其中一个键的值是一个BaseMessage序列

（3）一个dict，其中一个键的值存储最后一次对话信息，另外一个键的值存储之前的历史对话信息

输出以下之一

（1）一个可以作为AIMessage的content的字符串

（2）一个BaseMessage序列

（3）一个dict，其中一个键的值是一个BaseMessage序列

首先需要一个返回BaseChatMessageHistory实例的可调用函数，这里我们将历史对话存储在内存中，同时langchain也支持将历史对话存储在redis中（RedisChatMessageHistory）更持久的存储，

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="你知道我的名字吗？")],
    config=config,
)
response

AIMessage(content='是的，你之前提到你的名字叫黑金。如果有其他问题或者需要帮助的地方，请告诉我。', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 140, 'total_tokens': 163}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-b61fa947-7287-4ee5-a117-74c432dbe32d-0')

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="我中午吃的是什么？")],
    config=config,
)
response

AIMessage(content='你之前提到，你中午吃的是米饭。如果你想讨论饮食或其他话题，请继续提问。', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 170, 'total_tokens': 191}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-9123e4cb-bdde-46a4-a2e9-48bd0b75a7f8-0')

#新的session_id
  'session_id':'id123'

In [ ]:
ret = with_message_history.invoke(input=HumanMessage("介绍下列宁"),config={'configurable':{'session_id':'id123'}})
ret

AIMessage(content='列宁（Vladimir Ilyich Lenin），原名弗拉基米尔·伊里奇·乌里扬诺夫（Vladimir Ilyich Ulyanov），是俄国革命的领导者之一，苏联的缔造者和第一任人民委员会主席（即苏联政府首脑）。他出生于1870年4月22日，逝世于1924年1月21日。\n\n列宁是一位马克思主义者，无产阶级革命家、政治家、理论家、思想家。他的理论和实践活动对20世纪的全球政治产生了深远的影响。以下是列宁的一些生平和事迹简介：\n\n### 早年生活\n列宁出生于沙皇俄国时期的辛比尔斯克（现乌里扬诺夫斯克），他的父亲是一位数学教育家，母亲是一位医生的女儿。列宁在学术上表现出色，但在1891年，他的哥哥亚历山大因参与刺杀沙皇亚历山大三世而被处决，这一事件对列宁的政治观点产生了重大影响。\n\n### 政治活动\n列宁在1893年成为马克思主义者，并很快成为俄国社会民主工党的活跃成员。1903年，该党分裂为布尔什维克（多数派）和孟什维克（少数派），列宁成为布尔什维克的领导者。\n\n### 列宁主义\n列宁在马克思主义的基础上发展了自己的理论，即列宁主义。他强调无产阶级专政、共产党的领导作用，以及在帝国主义时代的革命策略。\n\n### 十月革命\n1917年，列宁领导了布尔什维克党发动的十月革命，推翻了临时政府，建立了世界上第一个社会主义国家——苏维埃俄国（后来成为苏联）。这一事件对全球政治和社会主义运动产生了深远的影响。\n\n### 执政生涯\n列宁在执政期间推行了一系列激进的政策，包括土地改革、国有化、战时共产主义政策等。他还创立了契卡（Cheka），即后来的克格勃（KGB），这是一个为了维护苏维埃政权而设立的安全机构。\n\n### 去世与遗产\n列宁于1924年因脑溢血去世。他的遗体经过防腐处理，安放在莫斯科红场上的列宁墓中。列宁的遗产，包括他的理论和实践，对苏联以及全球的共产主义运动产生了持久的影响。\n\n列宁是一个极具争议的历史人物，被一些人视为伟大的革命家和无产阶级的领袖，同时也被另一些人批评为独裁者和极权主义的代表。不管怎样，他无疑是20世纪最重要的政治人物之一。', response_metadata={'token_usage': {'completion_tokens': 495, 'prompt_t

In [ ]:
ret = with_message_history.invoke(input=HumanMessage("介绍下名人胡志明"),config={'configurable':{'session_id':'id123'}})
ret

AIMessage(content='胡志明（Hồ Chí Minh，1890年5月19日 - 1969年9月2日），原名阮必成（Nguyễn Sinh Cung），是越南著名的共产主义革命家、政治家，越南民主共和国的主要创立者和越南劳动党（今越南共产党）的创建者之一。他曾在1946年至1969年期间担任越南民主共和国的主席，并兼任政府总理。\n\n胡志明出生于越南乂安省的一个儒生家庭，他的祖父和父亲都曾做过官。胡志明早年接受了良好的教育，学习了汉语和儒家经典。在青年时期，他曾经当过教师、海员和杂役，这些经历使他对社会现实有了深刻的认识。\n\n1920年，胡志明在法国加入了法国共产党，开始了他的政治生涯。1925年，他在中国广州组织了越南青年革命同志会。1930年，在香港领导成立了越南共产党（后改名为印度支那共产党、越南劳动党）。在第二次世界大战期间，胡志明在中国参与了抗日战争，并积极组织越南国内反对法国殖民者的斗争。\n\n1941年，胡志明返回越南，成立了越南独立同盟会（简称越盟），领导反对法国维希政府和日本侵略者的斗争。1945年8月，胡志明领导了越南八月革命，成功推翻了日本和法国的殖民统治，并在9月2日宣布越南民主共和国的成立。\n\n胡志明在越南独立后的建设和发展中扮演了重要角色，他致力于国家的统一和社会主义建设。胡志明也是越南与中国关系的重要推动者，与中国的毛泽东、周恩来等领导人建立了深厚的友谊。\n\n胡志明思想是越南共产党官方意识形态的重要组成部分，被认为是马克思主义在越南的具体应用。他的思想在1991年被正式写入越南共产党的官方文件。\n\n胡志明去世后，越南最大的城市西贡被改名为胡志明市，以纪念他的贡献。胡志明在越南被广泛尊敬，被视为民族英雄和伟大的领导者。他的生日也被联合国教科文组织建议作为纪念活动日。然而，胡志明的一生和遗产也受到了一些批评和争议，特别是在海外越南社区中。', response_metadata={'token_usage': {'completion_tokens': 457, 'prompt_tokens': 1714, 'total_tokens': 2171}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-74452376-7d9b-4010-85

In [ ]:
ret = with_message_history.invoke(input=HumanMessage("请问我之前要求你介绍过哪些名人"),config={'configurable':{'session_id':'id123'}})
ret

AIMessage(content='根据我们的对话记录，您之前要求我介绍过的名人是：\n\n1. 胡志明（Hồ Chí Minh） - 越南的共产主义革命家和政治家。\n2. 列宁（Vladimir Ilyich Lenin） - 俄国革命的领导者之一，苏联的缔造者和第一任人民委员会主席。\n\n如果您需要关于其他名人的介绍，请随时告诉我。', response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 1168, 'total_tokens': 1253}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-889a2741-a9f6-49a1-a273-3095ae011bb0-0')

#说明：
1， 提示：WARNING:langchain_core.tracers.core:Parent run 412b2986-8016-4271-8952-ef5ee2e50e22 not found for run 889a2741-a9f6-49a1-a273-3095ae011bb0. Treating as a root run.
这个不影代码有效性。，只要注意提示方式

如：我是黑金。问AI，你知道我的名字吗？ AI是无法从历史记录内找到并回复的